In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os
os.chdir('/home/hrai/codes/PoseAdaptor')
from lib_import import *
from my_utils import *

# 주요 디렉토리 경로
user = getpass.getuser()
alphapose_root = '/home/{}/codes/AlphaPose'.format(user)
motionbert_root = '/home/{}/codes/MotionBERT'.format(user)
kookmin_root = '/home/{}/Datasets/HAAI/국민대데이터/data'.format(user)
ap_kookmin_result_root = alphapose_root + "/examples/kookmin_result2"

assert os.path.isdir(alphapose_root), "AlphaPose root directory is not exist"
assert os.path.isdir(motionbert_root), "MotionBERT root directory is not exist"
assert os.path.isdir(kookmin_root), "Kookmin root directory is not exist"
assert os.path.isdir(ap_kookmin_result_root), "AlphaPose Kookmin result directory is not exist"

alphapose_root, motionbert_root, kookmin_root, ap_kookmin_result_root

('/home/hrai/codes/AlphaPose',
 '/home/hrai/codes/MotionBERT',
 '/home/hrai/Datasets/HAAI/국민대데이터/data',
 '/home/hrai/codes/AlphaPose/examples/kookmin_result2')

### Load pyskl result

In [23]:
pyskl_data = readpkl('/home/hrai/codes/MotionBERT/data/action/kookmin_hrnet_modified.pkl')
pyskl_data.keys()

dict_keys(['split', 'annotations'])

In [27]:
pyskl_data['split'].keys(), pyskl_data['split']['train'][:3], pyskl_data['split']['test'][:3]

(dict_keys(['train', 'test']),
 ['train_S09_burpee_2', 'train_S07_butterfly_1', 'train_S04_pushup_7'],
 ['test_S08_burpee_4', 'test_S08_kneeup_1', 'test_S08_squat_2'])

In [70]:
len(pyskl_data['split']['train'])

389

In [51]:
len(pyskl_data['annotations']), pyskl_data['annotations'][0].keys()

(557,
 dict_keys(['frame_dir', 'img_shape', 'num_person_raw', 'keypoint', 'keypoint_score', 'total_frames', 'label']))

In [83]:
i = -5
for key in pyskl_data['annotations'][i].keys():
    if type(pyskl_data['annotations'][i][key]) == np.ndarray:
        print(key, pyskl_data['annotations'][i][key].shape)
    else:
        print(key, pyskl_data['annotations'][i][key])

frame_dir test_S08_benchpress_3
img_shape (1920, 1080)
num_person_raw 6
keypoint (1, 959, 17, 2)
keypoint_score (1, 959, 17)
total_frames 959
label 5


### Load mmpose result

In [30]:
root = '/home/hrai/codes/mmpose/work_dirs/rtmpose-l_8xb256-420e_kookmin2-256x192/kookmin2_epoch50'
result_list = glob(root + '/*.json')
result_list[:5]

['/home/hrai/codes/mmpose/work_dirs/rtmpose-l_8xb256-420e_kookmin2-256x192/kookmin2_epoch50/results_S18_burpee_1.json',
 '/home/hrai/codes/mmpose/work_dirs/rtmpose-l_8xb256-420e_kookmin2-256x192/kookmin2_epoch50/results_S12_pushup_1.json',
 '/home/hrai/codes/mmpose/work_dirs/rtmpose-l_8xb256-420e_kookmin2-256x192/kookmin2_epoch50/results_S18_cat_4.json',
 '/home/hrai/codes/mmpose/work_dirs/rtmpose-l_8xb256-420e_kookmin2-256x192/kookmin2_epoch50/results_S15_kneeup_5.json',
 '/home/hrai/codes/mmpose/work_dirs/rtmpose-l_8xb256-420e_kookmin2-256x192/kookmin2_epoch50/results_S18_cat_3.json']

In [33]:
mmpose_data = readJSON(result_list[0])
mmpose_data.keys()

dict_keys(['meta_info', 'instance_info'])

In [35]:
mmpose_data['meta_info'].keys()

dict_keys(['dataset_name', 'num_keypoints', 'keypoint_id2name', 'keypoint_name2id', 'upper_body_ids', 'lower_body_ids', 'flip_indices', 'flip_pairs', 'keypoint_colors', 'num_skeleton_links', 'skeleton_links', 'skeleton_link_colors', 'dataset_keypoint_weights', 'sigmas', 'CLASSES'])

In [50]:
for key in mmpose_data['meta_info'].keys():
    print(key, mmpose_data['meta_info'][key])

dataset_name fit3d
num_keypoints 17
keypoint_id2name {'0': 'pelvis', '1': 'right_hip', '2': 'right_knee', '3': 'right_ankle', '4': 'left_hip', '5': 'left_knee', '6': 'left_ankle', '7': 'torso', '8': 'neck', '9': 'nose', '10': 'head', '11': 'left_shoulder', '12': 'left_elbow', '13': 'left_wrist', '14': 'right_shoulder', '15': 'right_elbow', '16': 'right_wrist'}
keypoint_name2id {'pelvis': 0, 'right_hip': 1, 'right_knee': 2, 'right_ankle': 3, 'left_hip': 4, 'left_knee': 5, 'left_ankle': 6, 'torso': 7, 'neck': 8, 'nose': 9, 'head': 10, 'left_shoulder': 11, 'left_elbow': 12, 'left_wrist': 13, 'right_shoulder': 14, 'right_elbow': 15, 'right_wrist': 16}
upper_body_ids [7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
lower_body_ids [0, 1, 2, 3, 4, 5, 6]
flip_indices [0, 4, 5, 6, 1, 2, 3, 7, 8, 9, 10, 14, 15, 16, 11, 12, 13]
flip_pairs [[4, 1], [5, 2], [6, 3], [1, 4], [2, 5], [3, 6], [14, 11], [15, 12], [16, 13], [11, 14], [12, 15], [13, 16]]
keypoint_colors {'__ndarray__': [[51, 153, 255], [255, 128, 0]

In [84]:
len(mmpose_data['instance_info']), mmpose_data['instance_info'][0].keys()

(1424, dict_keys(['frame_id', 'instances']))

In [49]:
mmpose_data['instance_info'][0]['frame_id'] # 

1

In [52]:
len(mmpose_data['instance_info'][0]['instances']), mmpose_data['instance_info'][0]['instances'][0].keys()

(5, dict_keys(['keypoints', 'keypoint_scores', 'bbox', 'bbox_score']))

In [91]:
for i in range(len(mmpose_data['instance_info'][0]['instances'])):
    keypoints = np.array(mmpose_data['instance_info'][0]['instances'][i]['keypoints'])
    keypoint_scores = np.array(mmpose_data['instance_info'][0]['instances'][i]['keypoint_scores'])
    bbox = np.array(mmpose_data['instance_info'][0]['instances'][i]['bbox'])
    bbox_score = np.array(mmpose_data['instance_info'][0]['instances'][i]['bbox_score'])
    print(keypoints.shape) 
    print(keypoint_scores.shape)
    print(bbox.shape)
    print(bbox_score)
    break

(17, 2)
(17,)
(1, 4)
1.0


In [ ]:
max_score = 0
max_idx = -1
for i in range(len(mmpose_result['instance_info'][frame_num]['instances'])):
    #print(np.mean(mmpose_result['instance_info'][frame_num]['instances'][i]['keypoint_scores']))
    if np.mean(mmpose_result['instance_info'][frame_num]['instances'][i]['keypoint_scores']) > max_score:
        max_score = np.mean(mmpose_result['instance_info'][frame_num]['instances'][i]['keypoint_scores'])
        max_idx = i
keypoint = np.array(mmpose_result['instance_info'][frame_num]['instances'][max_idx]['keypoints'])

### mmpose to pyskl

In [108]:
def get_label_from_video_name(video_name):
    if 'squat' in video_name: label = 0
    elif 'burpee' in video_name: label = 1
    elif 'pushup' in video_name: label = 2
    elif 'kneeup' in video_name: label = 3
    elif 'yoga' in video_name: label = 4
    elif 'benchpress' in video_name: label = 5
    elif 'cat' in video_name: label = 6
    return label

In [111]:
root = '/home/hrai/codes/mmpose/work_dirs/rtmpose-l_8xb256-420e_kookmin2-256x192/kookmin2_epoch50'
result_list = natsorted(glob(root + '/*.json'))

pyskl_from_mmpose = {}
pyskl_from_mmpose['split'] = {'train': [], 'test': []}
pyskl_from_mmpose['annotations'] = []

train_subjects = ['S11', 'S12']

for item in tqdm(result_list): # for each video
    #print(item)
    # load mmpose result
    mmpose_result = readJSON(item)
    
    # get video name
    video_name = os.path.basename(item).split('.')[0].split('results_')[1]
    #print(video_name)
    
    # get subject
    subject, action, _ = video_name.split('_')
    label = get_label_from_video_name(video_name)
    
    # split
    if subject in train_subjects: split = 'train'
    else: split = 'test'
    total_frames = len(mmpose_result['instance_info'])
    pyskl_from_mmpose['split'][split].append(video_name)
    keypoint = []
    keypoint_score = []
    for frame_num in range(total_frames):
        max_score = 0
        max_idx = -1
        for i in range(len(mmpose_result['instance_info'][frame_num]['instances'])):
            if np.mean(mmpose_result['instance_info'][frame_num]['instances'][i]['keypoint_scores']) > max_score:
                max_score = np.mean(mmpose_result['instance_info'][frame_num]['instances'][i]['keypoint_scores'])
                max_idx = i
        keypoint.append(np.array(mmpose_result['instance_info'][frame_num]['instances'][max_idx]['keypoints']))
        keypoint_score.append(np.array(mmpose_result['instance_info'][frame_num]['instances'][max_idx]['keypoint_scores']))
    
    video_result = {}
    video_result['frame_dir'] = video_name
    video_result['img_shape'] = (1920, 1080)
    video_result['num_person_raw'] = 1
    video_result['keypoint'] = np.array([keypoint])
    video_result['keypoint_score'] = np.array([keypoint_score])
    video_result['total_frames'] = total_frames
    video_result['label'] = label
    
    # print(video_result['frame_dir'])
    # print(np.array(video_result['keypoint']).shape)
    # print(np.array(video_result['keypoint_score']).shape)
    
    pyskl_from_mmpose['annotations'].append(video_result)
    #break

100%|██████████| 350/350 [01:08<00:00,  5.12it/s]


In [112]:
savepkl(pyskl_from_mmpose, '/home/hrai/codes/MotionBERT/data/action/kookmin2_mmpose_train_s11-12.pkl')